In [2]:
import facebook
import requests
from bs4 import BeautifulSoup as BS
import os
import random as rnd
from string import Template
from collections import namedtuple
import csv

utm_params_string = "${website_url}?utm_source=${campaign_source}&utm_medium=${campaign_medium}&utm_campaign=${campaign_name}&utm_term=${campaign_term}&utm_content=${campaign_content}"

TemplateType = namedtuple("TemplateTuple","type template utm")
Templates = {
    11:TemplateType("price11",Template("Hotel ${hotel} za \U0001F4A3 \N{bomb} bombovou cenu ${price_c} s odletem \U0001F1EA ${date_from}. Trvání ${duration}, cena za den ${price_oneday}."),"utm"),
    12:TemplateType("price12",Template("Hotel ${hotel} za \U0001F4A3 bombovou cenu ${price_c} s odletem \N{airplane} ${date_from}."),"utm"),
    2:TemplateType("price2",Template("Odleťte už ${date_from} do hotelu ${hotel} za úžasnou cenu ${price_c}."),"utm"),
    3:TemplateType("price3",Template("Zaplaťte jen ${price_c} a navštivte ${hotel} od ${date_from}."),"utm"),
    4:TemplateType("price4",Template("Za ${price_c} od ${date_from} do ${rating} hotelu ${hotel} do prosluněné destinace ${destination}."),"utm"),
    5:TemplateType("price5",Template("${country}, ${destination} do ${rating} hotelu ${hotel} v termínu ${date_whole} za ${price_f}."),"utm")
}

my_pages=[
    {"page_id":"633916606749334","page_title":"Cestuj hned","page_user_name":"@travelasap.cz"},
    {"page_id":"1645656035685167","page_title":"Travelasap","page_user_name":"@cestujhned"}
]
fb_app={"access_token":"EAAEmaBNOAYYBANbCLqByui1QksGsq77TyuUYfn5GCyJdVeDTEU0HJvgChZAZBuqMWk0lDFSfPP7oyiWHYASl2Iga2SASETDv6ZC2dnoAEJgY3XUsGgOvBAzTaVJFsFLZAsZARwf7NgczSSZCEZBXZBufyX5zoZAxu8UfvH8qcfyyUlwZDZD",
"app_id":"323703419044230",
"app_sec":"fc4558854f4718fd420c7adeeb3c7798"}

ta_url = "https://www.travelasap.cz/d/"

trip_nos=[207767334,211750419,214387513,219445361]

#print("Verze facebook-sdk:",facebook.__version__)
def post_to_fb(msg, trip_param):
    #(page_url,title,hotel,hotel2,board,price,price2,duration,date_whole,date_from,date_to,discount,pictures) = trip_param
    # Fill in the values noted in previous steps here
    cfg = {
    "page_id"      : my_pages[0]["page_id"],
    "access_token" : fb_app["access_token"]
    }

    api = get_api(cfg)
    
    status = api.put_object(
      parent_object="me",
      connection_name="feed",
      message=msg,
      link=trip_param["url_full"])


"""
My "app" allows our fans to find great travel opportunities. Every trip has a lot of parameters according to which the fans decide, so I don't want to make the mistake of "manual rewriting". We are preparing for invoiced campaigns on Facebook, so we must be accurate and interesting in terms of content. We will also use the script to add UTM parameters to the URL.
"""

def get_api(cfg):
  graph = facebook.GraphAPI(cfg['access_token'])
  # Get page token to post as the page. You can skip 
  # the following if you want to post as yourself. 
  resp = graph.get_object('me/accounts')
  page_access_token = None
  for page in resp['data']:
    if page['id'] == cfg['page_id']:
      page_access_token = page['access_token']
  graph = facebook.GraphAPI(page_access_token)
  return graph

def get_msg(trip_no, trip_param, msg_type="price1"):
  msg_type_index = 11
  #msg_type_index = rnd.randint(1,len(Templates))
  msg = Templates[msg_type_index].template.substitute(trip_param)
  return msg

def get_params_from_ta(trip_no):
  trip_params = {}
  status=0
  page_url = os.path.join(ta_url,str(trip_no))
  #print(page_url)
  while status!=200:
    page = requests.get(page_url)
    status = page.status_code
  soup = BS(page.content, 'html.parser')

  # Tady naparsuju všechny parametry, které potřebuju, a přiřadím do slovníku s parametry
  # Tady naparsuju všechny parametry, které potřebuju, a přiřadím do slovníku s parametry
  trip_params["trip_no"] = str(trip_no)
  trip_params["url"] = page_url
  trip_params["url_full"] = page.url.split(sep="?")[0]
  trip_params["title"] = soup.find("h1").text.strip()
  trip_params["hotel"] = soup.find("div", class_='hotel').text.strip()
  trip_params["hotel2"] = trip_params["title"].split(" za cenu ")[0].strip()
  trip_params["board"] = soup.find("div", class_='board').text.strip()
  trip_params["price_f"] = soup.find("a", class_="change-structured-content pric b-pric").text.strip().replace(" ","") #cena včetně nedělitelných mezer
  trip_params["price_c"] = trip_params["title"].split(" ")[-5] #cena kompaktní bez mezer, ale s jednotkou měny
  trip_params["price_n"] = int(trip_params["price_c"][:-2])
  trip_params["duration"] = soup.find("div", class_='dur').text.strip()
  trip_params["price_oneday"] = int(trip_params["price_n"]/int(trip_params["duration"].split(" ")[0]))
  trip_params["date_whole"] = soup.find("div", class_='date').text.strip()
  #date_from = soup.find("div", class_='date').text.strip()
  #date_to = soup.find("div", class_='date').text.strip()
  trip_params["destination"] = soup.find("div", class_='dest').text.strip()
  trip_params["country"] = trip_params["destination"].split(sep="»")[0].strip()
  trip_params["rating"] = "5*"
  trip_params["date_from"] = trip_params["title"].split(" ")[-3]
  trip_params["date_to"] = trip_params["title"].split(" ")[-1]
  trip_params["airport"] = soup.find("div", class_='air').text.strip()
  trip_params["discount"] = soup.find("div", class_='dis').text.strip()
  trip_params["pictures"] = ["https:"+a['href'] for a in soup.find("div", class_='pics-inner').find_all('a', href=True) if a.text]
  trip_params["rating"] = str(len(soup.find(dir,class_="rating").find_all("img", class_="nb star")))+"*"
  
  return trip_params



def main():
  for trip_no in trip_nos:
    trip_param = get_params_from_ta(trip_no)
    print(trip_param)
    #(title,hotel,hotel2,board,price,price2,duration,date_whole,date_from,date_to,discount,pictures) = get_params_from_ta(trip_no)
    #print(title,hotel,hotel2,board,price,price2,duration,date_whole,date_from,date_to,discount,pictures)
    #print(trip_param)
    message = get_msg(trip_no,trip_param)
    print(message)
    #post_to_fb(message, trip_param)

main()

{'trip_no': '207767334', 'url': 'https://www.travelasap.cz/d/207767334', 'url_full': 'https://www.travelasap.cz/chcete-jet-do/bella-vista-hotel-resort/egypt/hurghada/7777a/207767334d', 'title': 'Bella Vista Hotel & Resort za cenu 13490Kč od 09.01.21 do 16.01.21', 'hotel': 'Bella Vista Hotel & Resort', 'hotel2': 'Bella Vista Hotel & Resort', 'board': 'All inclusive', 'price_f': '13\xa0490\xa0Kč', 'price_c': '13490Kč', 'price_n': 13490, 'duration': '8 dní', 'price_oneday': 1686, 'date_whole': '9.1. - 16.1.2021', 'destination': 'Egypt » Hurghada', 'country': 'Egypt', 'rating': '4*', 'date_from': '09.01.21', 'date_to': '16.01.21', 'airport': 'Praha', 'discount': '-0 %', 'pictures': ['https://i.ccdn.cz/acm/59/155420/l0.jpg', 'https://i.ccdn.cz/acm/59/155420/l1.jpg', 'https://i.ccdn.cz/acm/59/155420/l2.jpg', 'https://i.ccdn.cz/acm/59/155420/l3.jpg', 'https://i.ccdn.cz/acm/59/155420/l4.jpg', 'https://i.ccdn.cz/acm/59/155420/l5.jpg', 'https://i.ccdn.cz/acm/59/155420/l6.jpg', 'https://i.ccdn.cz